In [15]:
import numpy as np
from collections import Counter

class DecisionNode:
    """Class that represents a decision node or leaf in the decision tree"""
    
    def __init__(self, feature_idx=None, threshold=None, left=None, right=None, *, value=None):
        self.feature_idx = feature_idx  # Index of feature to split on
        self.threshold = threshold      # Threshold value for the split
        self.left = left                # Left subtree (<= threshold)
        self.right = right             # Right subtree (> threshold)
        self.value = value              # Value if leaf node (class label)
    
    def is_leaf(self):
        """Check if node is a leaf"""
        return self.value is not None

In [16]:
class DecisionTree:
    """Decision Tree Classifier from scratch"""
    
    def __init__(self, max_depth=100, min_samples_split=2):
        self.max_depth = max_depth          # Maximum depth of the tree
        self.min_samples_split = min_samples_split  # Minimum samples to split
        self.root = None                    # Root node of the tree
    
    def fit(self, X, y):
        """Build the decision tree"""
        self.n_classes = len(np.unique(y))  # Number of classes
        self.n_features = X.shape[1]        # Number of features
        self.root = self._grow_tree(X, y)
    
    def _grow_tree(self, X, y, depth=0):
        """Recursively grow the decision tree"""
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        # Stopping criteria
        if (depth >= self.max_depth 
            or n_labels == 1 
            or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return DecisionNode(value=leaf_value)
        
        # Find best split
        best_feature, best_thresh = self._best_split(X, y)
        
        # If no split improves purity, create leaf node
        if best_feature is None:
            return DecisionNode(value=self._most_common_label(y))
        
        # Split the data
        left_idxs = X[:, best_feature] <= best_thresh
        right_idxs = X[:, best_feature] > best_thresh
        left = self._grow_tree(X[left_idxs], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs], y[right_idxs], depth+1)
        
        return DecisionNode(best_feature, best_thresh, left, right)
    
    def _best_split(self, X, y):
        """Find the best split for a node"""
        best_gain = -1
        best_feature, best_thresh = None, None
        
        # Try all features
        for feature_idx in range(self.n_features):
            X_column = X[:, feature_idx]
            thresholds = np.unique(X_column)
            
            # Try all thresholds for current feature
            for threshold in thresholds:
                # Calculate information gain
                gain = self._information_gain(y, X_column, threshold)
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_idx
                    best_thresh = threshold
        
        return best_feature, best_thresh
    
    def _information_gain(self, y, X_column, threshold):
        """Calculate information gain of a split"""
        # Parent entropy
        parent_entropy = self._entropy(y)
        
        # Split data
        left_idxs = X_column <= threshold
        right_idxs = X_column > threshold
        
        if len(y[left_idxs]) == 0 or len(y[right_idxs]) == 0:
            return 0
        
        # Weighted average child entropy
        n = len(y)
        n_left, n_right = len(y[left_idxs]), len(y[right_idxs])
        e_left, e_right = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_left/n) * e_left + (n_right/n) * e_right
        
        # Information gain is difference in entropy
        info_gain = parent_entropy - child_entropy
        return info_gain
    
    def _entropy(self, y):
        """Calculate entropy of a set of labels"""
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])
    
    def _most_common_label(self, y):
        """Find the most common label in a set"""
        counter = Counter(y)
        return counter.most_common(1)[0][0]
    
    def predict(self, X):
        """Predict class for input samples"""
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def _traverse_tree(self, x, node):
        """Traverse the tree to make a prediction"""
        if node.is_leaf():
            return node.value
        
        if x[node.feature_idx] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [17]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train decision tree
tree = DecisionTree(max_depth=10)
tree.fit(X_train, y_train)

# Make predictions
predictions = tree.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00
